# sktime-xai: ROCKET Interpretability Demo

This notebook demonstrates how to peer inside the `RocketClassifier` black box.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sktime.datasets import load_gunpoint
from sktime.classification.kernel_based import RocketClassifier
from sktime_xai.classification._rocket_explainer import RocketExplainer

## 1. Train ROCKET on GunPoint
GunPoint is a classic dataset: distinguish between drawing a gun vs. pointing a finger.

In [ ]:
X_train, y_train = load_gunpoint(split="train")
X_test, y_test = load_gunpoint(split="test")

# Using a small number of kernels for speed and easier demo
clf = RocketClassifier(num_kernels=500, random_state=42)
clf.fit(X_train, y_train)
print("Training complete.")

## 2. Explain a Prediction
Let's see why the model classified the first test instance as such.

In [ ]:
# Select a test instance
idx = 0
X_instance = X_test.iloc[[idx]]
true_class = y_test[idx]
print(f"Instance {idx}, True Class: {true_class}")

explainer = RocketExplainer(clf)

# In GunPoint, classes are '1' and '2'. 
# RocketClassifier maps them to internal indices.
# Let's simply explain Class index 0 (or 1 depending on prediction).
# We can ask the explainer to explain the *predicted* class.

pred_class_idx = 0 # Just assuming index 0 for demo simplicity

explanation = explainer.explain(X_instance, class_index=pred_class_idx, top_k=3)

fig = explanation.plot()
plt.show()

**Interpretation:**
The **Red Line** shows the convolution activation of the kernel.
The **Gray Line** is the original signal.
- If Feature Type is `Max`, the **Black Star** shows the max activation point that triggered the feature.
- If Feature Type is `PPV`, the kernel contributes based on how much of the red line is positive.